# *Pump-it-up project*

### Can you predict which water pumps are faulty?

## Goal
Using data from Taarifa and the Tanzanian Ministry of Water, predict which pumps are functional, which need some repairs, and which don't work at all based on a number of variables about what kind of pump is operating, when it was installed, and how it is managed. 

A smart understanding of which waterpoints will fail can improve maintenance operations and ensure that clean, potable water is available to communities across Tanzania.

# II. Data Cleaning & Preprocessing
## 1. Libraries and input data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline

print("Setup Complete")

In [ ]:
# Load the data after EDA
train_df = pd.read_csv("../input/pumpitup-challenge-dataset/train_df_after_EDA.csv")
X_test = pd.read_csv("../input/pumpitup-challenge-dataset/X_test_after_EDA.csv")

## 2. Dropping similar features

The following groups of features

- (extraction_type, extraction_type_group, extraction_type_class),
- (payment, payment_type),
- (water_quality, quality_group),
- (source, source_class),
- (subvillage, region, region_code, district_code, lga, ward),
- (waterpoint_type, waterpoint_type_group)
- (scheme_name, scheme_management)

contain very similar information, so the correlation between them is high. This way we are risking overfitting the training data by including all the features in our analysis.

Besides:

num_private is ~99% zeros and has no description, so we cannot interpret it
in the wpt_name feature 45k unique values out of 75k observations, not very informative -> drop for now

**TO DO**: The correlation between "construction_year" and "gps_height" is high, but these 2 variables don't have any obvious connection, so explore this correlation further to take a decision.

As we saw earlier, there exists quite a strong correlation between district_code and region_code, so we will drop one of these variables. The negative correlation to the target variable of the "region_code" is higher than that of the "district_code". Keep the variable with higher correlation to the target.

In [ ]:
# drop columns
train_df = train_df.drop(['installer','management_group','status_group','id_x','id_y', 'num_private', 'wpt_name', 
          'recorded_by', 'subvillage', 'scheme_name', 'region', 
          'quantity', 'water_quality', 'lga','ward', 'source_type', 'payment', 
          'waterpoint_type_group','extraction_type_group','extraction_type_class'],axis=1)
X_test = X_test.drop(['installer','management_group','id', 'num_private', 'wpt_name', 
          'recorded_by', 'subvillage', 'scheme_name', 'region', 
          'quantity', 'water_quality', 'lga','ward', 'source_type', 'payment', 
          'waterpoint_type_group','extraction_type_group','extraction_type_class'],axis=1)

In [ ]:
train_df.head()

## 3. Dealing with missing values

In [ ]:
## Null scheme_management, funder, installer, public_meeting and permit values replace with "unknown" text
train_df["scheme_management"].fillna("unknown", inplace = True)
train_df["public_meeting"].fillna("unknown", inplace = True)
train_df["permit"].fillna("unknown", inplace = True)
train_df["funder"].fillna("unknown", inplace = True)
# train_df["installer"].fillna("unknown", inplace = True)

X_test["scheme_management"].fillna("unknown", inplace = True)
X_test["public_meeting"].fillna("unknown", inplace = True)
X_test["permit"].fillna("unknown", inplace = True)
X_test["funder"].fillna("unknown", inplace = True)
# X_test["installer"].fillna("unknown", inplace = True)

In [ ]:
X_test.isna().sum().sort_values(ascending=False)

## ** Transforming data to reduce skew
Uni-modal, skewed distributions could potentially be log transformed: 
* Longtitude
* GPS_hight
* Region_code

** For later

## 5. Reducing cardinality
### 5.1 Select features

In [ ]:
# Get number of unique entries in each column with categorical data
cat_vars = train_df.select_dtypes(include='object').columns
object_nunique = list(map(lambda col: train_df[col].nunique(), cat_vars))
d = dict(zip(cat_vars, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

Based on the above analysis:

the "recorded_by" feature can be dropped as there is only 1 unique value, it doesn't help in predicting.

The columns in which values can be ordered we can perform an Ordinal encoding: 
* quality_group
* quantity_group
* payment_type

The cardinality of the following 2 features should be reduced to 10 and then one-hot encode them or try Binary encoding:
* scheme_managenemt
* extraction_type

What to do with the following 3? The cardinality is too high... : --> will drop at the first model run, later could try Frequency encoding,Binary encoding if reduce cardinality to at least 100.
* funder
* installer
* subvillage

The rest can be one-hot encoded as the cardinality is lower than 10:
* public_meeting # later -> Binary?
* permit # later-> Binary?
* source_class
* management_group
* waterpoint_type_group
* source_type
* basin

### 5.2 Scheme_management

In [ ]:
train_df.scheme_management.value_counts()

In [ ]:
## scheme_management
def scheme_wrangler(row):
    if row['scheme_management']=='VWC':
        return 'vwc'
    elif row['scheme_management']=='WUG':
        return 'wug'
    elif row['scheme_management']=='Water authority':
        return 'wtr_auth'
    elif row['scheme_management']=='WUA':
        return 'wua'
    elif row['scheme_management']=='Water Board':
        return 'wtr_brd'
    elif row['scheme_management']=='Parastatal':
        return 'parastatal'
    elif row['scheme_management']=='Private operator':
        return 'pri_optr'
    elif row['scheme_management']=='SWC':
        return 'swc'
    elif row['scheme_management']=='Company':
        return 'company'
    else:
        return 'other'
train_df['scheme_management'] = train_df.apply(lambda row: scheme_wrangler(row), axis=1)
X_test['scheme_management'] = X_test.apply(lambda row: scheme_wrangler(row), axis=1)

### 5.3 Extraction_type

In [ ]:
train_df.extraction_type.value_counts()

In [ ]:
## extraction_type
def extraction_wrangler(row):
    if row['extraction_type']=='gravity':
        return 'gravity'
    elif row['extraction_type']=='nira/tanira':
        return 'nira/tanira'
    elif row['extraction_type']=='submersible':
        return 'submersible'
    elif row['extraction_type']=='swn 80':
        return 'swn_80'
    elif row['extraction_type']=='mono':
        return 'mono'
    elif row['extraction_type']=='india mark ii':
        return 'india_mark_ii'
    elif row['extraction_type']=='afridev':
        return 'afridev'
    elif row['extraction_type']=='ksb':
        return 'ksb'
    elif row['extraction_type']=='windmill':
        return 'windmill'
    else:
        return 'other'
train_df['extraction_type'] = train_df.apply(lambda row: extraction_wrangler(row), axis=1)
X_test['extraction_type'] = X_test.apply(lambda row: extraction_wrangler(row), axis=1)

### 5.4 Funder

In [ ]:
train_df.funder.value_counts()

In [ ]:
## funder
def funder_wrangler(row):  
    '''Keep top 8 values and set the rest to 'other'''

    if row['funder']=='Government Of Tanzania':
        return 'gov'
    elif row['funder']=='Danida':
        return 'danida'
    elif row['funder']=='Hesawa':
        return 'hesawa'
    elif row['funder']=='Rwssp':
        return 'rwssp'
    elif row['funder']=='World Bank':
        return 'world_bank'   
    elif row['funder']=='Kkkt':
        return 'kkkt'   
    elif row['funder']=='World Vision':
        return 'world_vision'  
    elif row['funder']=='Unicef':
        return 'unicef'
    else:
        return 'other'
    
train_df['funder'] = train_df.apply(lambda row: funder_wrangler(row), axis=1)
X_test['funder'] = X_test.apply(lambda row: funder_wrangler(row), axis=1)

## 6. Ordinal encoding of categorical data
Several categorical features can be encoded in a specific order that follows from the range of its values. By using ordinal encoding instead of one-hot encoding we will avoid creating numerious additional columns and provide some logic to the model on how to evaluate these features. For example for the quality_group variable, the higher the label, the better the water quality, the more likely a pump is functional.

### 6.1 Quality_group

In [ ]:
train_df.quality_group.value_counts()

In [ ]:
order_dict_quality = {"good":3,"salty":2,"milky":2,"colored":2,"fluoride":2,"unknown":1}
train_df["quality_group_code"] = [order_dict_quality[item] for item in train_df.quality_group]
del train_df["quality_group"]

X_test["quality_group_code"] = [order_dict_quality[item] for item in X_test.quality_group]
del X_test["quality_group"]

### 6.2 Quantity_group

In [ ]:
train_df.quantity_group.value_counts()

In [ ]:
order_dict_quantity = {"enough":3,"insufficient":2,"dry":2,"seasonal":2,"unknown":1}
train_df["quantity_group_code"] = [order_dict_quantity[item] for item in train_df.quantity_group] 
del train_df["quantity_group"]

X_test["quantity_group_code"] = [order_dict_quantity[item] for item in X_test.quantity_group] 
del X_test["quantity_group"]

### 6.3 Payment_type

In [ ]:
train_df.payment_type.value_counts()

In [ ]:
order_dict_payment = {"monthly":4,"annually":4,"on failure":3,"per bucket":3,"never pay":2,"unknown":1,"other":1}
train_df["payment_code"] = [order_dict_payment[item] for item in train_df.payment_type] 
del train_df["payment_type"]

X_test["payment_code"] = [order_dict_payment[item] for item in X_test.payment_type] 
del X_test["payment_type"]

### 6.4 Public_meeting

In [ ]:
train_df.public_meeting.value_counts()

In [ ]:
order_dict_pub_meet = {True:2,False:1,"unknown":0}
train_df["public_meeting_code"] = [order_dict_pub_meet[item] for item in train_df.public_meeting] 
del train_df["public_meeting"]

X_test["public_meeting_code"] = [order_dict_pub_meet[item] for item in X_test.public_meeting] 
del X_test["public_meeting"]

### 6.5 Permit

In [ ]:
train_df.permit.value_counts()

In [ ]:
order_dict_permit = {True:2,False:1,"unknown":0}
train_df["permit_code"] = [order_dict_pub_meet[item] for item in train_df.permit] 
del train_df["permit"]

X_test["permit_code"] = [order_dict_pub_meet[item] for item in X_test.permit] 
del X_test["permit"]

## 7. Feature Engineering

Feature engineering is a powerful way to improve the model. We will create new variables based on the features in the dataset that will better describe the target.

### 7.1 Amount_tsh
Based on the EDA we have previously defined a threshold to separate functional and non-functional pumps. Let's create a new binary variable that reflects this information.

In [ ]:
train_df.loc[train_df['amount_tsh'] < 200000, 'amount_tsh'] = 0
train_df.loc[train_df['amount_tsh'] >= 200000, 'amount_tsh'] = 1

X_test.loc[train_df['amount_tsh'] < 200000, 'amount_tsh'] = 0
X_test.loc[train_df['amount_tsh'] >= 200000, 'amount_tsh'] = 1

### 7.2 Combine longitude and latitude in one feature
In this section we tried to create a new variable that would combine both longitude and latitude using a zip method. 

Yellow spots - functional pumps, green - needs repair, purple - non-functional.

As we see, south-est and almost all central and south west pumps are mostly non-functional. Area around the capital (central-south part) is mostly functional as well as north-est pumps. The rest seems unclear. 

How to use this info for the model? Try to create clusters (for ex around big cities)?

I have tested this featureand realized that models are unable to process a tuple. We will use the linear discriminant analysis for these variables.

### 7.3 Linear Discriminant Analysis (LDA)

LDA helps reducing dimentiality by maximizing the difference between categories. In our highly dimentional model it is a great tool. We will use LDA for the following columns as all of them contain geographical information:
 
- latitude
- longitude
- gps_height

Reference: https://zlatankr.github.io/posts/2017/01/23/pump-it-up

In [ ]:
LDA_cols = ["latitude","longitude","gps_height"]

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

train_df_sc = sc.fit_transform(train_df[LDA_cols])
X_test_sc = sc.transform(X_test[LDA_cols])

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components=None)

train_df_lda = lda.fit_transform(train_df_sc, train_df.label.values.ravel())
X_test_lda = lda.transform(X_test_sc)

train_df = pd.concat((pd.DataFrame(train_df_lda), train_df), axis=1)
X_test = pd.concat((pd.DataFrame(X_test_lda), X_test), axis=1)
    
    
for i in LDA_cols:
    del train_df[i]
    del X_test[i]

In [ ]:
train_df.rename(columns={0: "LDA_0",1:"LDA_1"},inplace=True)
X_test.rename(columns={0: "LDA_0",1:"LDA_1"},inplace=True)

### 7.4 Construction year
We will turn construction_year into a categorical column with bins containing the following values: '60s', '70s', '80s', '90s, '00s', '10s', 'unknown'.

In [ ]:
def construction_wrangler(row):
    if row['construction_year'] >= 1960 and row['construction_year'] < 1970:
        return '60s'
    elif row['construction_year'] >= 1970 and row['construction_year'] < 1980:
        return '70s'
    elif row['construction_year'] >= 1980 and row['construction_year'] < 1990:
        return '80s'
    elif row['construction_year'] >= 1990 and row['construction_year'] < 2000:
        return '90s'
    elif row['construction_year'] >= 2000 and row['construction_year'] < 2010:
        return '00s'
    elif row['construction_year'] >= 2010:
        return '10s'
    else:
        return 'unknown'
    
train_df['construction_year'] = train_df.apply(lambda row: construction_wrangler(row), axis=1)
X_test['construction_year'] = X_test.apply(lambda row: construction_wrangler(row), axis=1)

### 7.5 Date_recorded
We will calculate the number of days past since the date_recorded of a particular pump till the most recent date of the dataset. The idea being that more recently recorded pumps might be more likely to be functional than non-functional. 

Let's first convert the column to type datetime. 

In [ ]:
train_df.date_recorded = pd.to_datetime(train_df.date_recorded)
X_test.date_recorded = pd.to_datetime(X_test.date_recorded)

train_df.date_recorded.describe()

In [ ]:
# The most recent data is 2013-12-03. Subtract each date from this point to obtain a 
# 'days_since_recorded' column.

train_df['days_since_recorded'] = pd.datetime(2013, 12, 3) - pd.to_datetime(train_df.date_recorded)
train_df['days_since_recorded'] = train_df['days_since_recorded'].astype('timedelta64[D]').astype(int)

X_test['days_since_recorded'] = pd.datetime(2013, 12, 3) - pd.to_datetime(X_test.date_recorded)
X_test['days_since_recorded'] = X_test['days_since_recorded'].astype('timedelta64[D]').astype(int)

In [ ]:
train_df['days_since_recorded']

In [ ]:
train_df = train_df.drop("date_recorded",axis=1)
X_test = X_test.drop("date_recorded",axis=1)

In [ ]:
train_df.shape

In [ ]:
X_test.shape

## 8. One-hot encoding of categorical features

In [ ]:
cat_vars = train_df.select_dtypes(include='object').columns
print(cat_vars)
len(cat_vars)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train_df[cat_vars])).astype(np.int64)
OH_cols_test = pd.DataFrame(OH_encoder.transform(X_test[cat_vars])).astype(np.int64)

# One-hot encoding removed index; put it back
OH_cols_train.index = train_df.index
OH_cols_test.index = X_test.index

OH_cols_train.columns = OH_encoder.get_feature_names(cat_vars)
OH_cols_test.columns = OH_encoder.get_feature_names(cat_vars)

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = train_df.drop(cat_vars, axis=1)
num_X_valid = X_test.drop(cat_vars, axis=1)

# Add one-hot encoded columns to numerical features
OH_train_df = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_test = pd.concat([num_X_valid, OH_cols_test], axis=1)

In [ ]:
OH_train_df.head()

## 9. Feature selection
### 9.1 L1 regularization with logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

X, y = OH_train_df[OH_train_df.columns.drop("label")], OH_train_df['label']

# Set the regularization parameter C=1
logistic = LogisticRegression(solver="saga",C=1, penalty="l1", random_state=7).fit(X, y)
model = SelectFromModel(logistic, prefit=True)

X_new = model.transform(X)
X_new

In [ ]:
# Get back the kept features as a DataFrame with dropped columns as all 0s
selected_features = pd.DataFrame(model.inverse_transform(X_new), 
                                 index=X.index,
                                 columns=X.columns)

# Dropped columns have values of all 0s, keep other columns 
selected_columns = selected_features.columns[selected_features.var() != 0]

In [ ]:
len(selected_columns)

In [ ]:
selected_columns

In [ ]:
train_df_selected_features = OH_train_df[selected_columns].join(y)

In [ ]:
X_test_selected_features = OH_X_test[selected_columns]

### 9.2 Feature importances with Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(criterion='gini',min_samples_split=8, n_estimators=1000,
                           random_state = 7)
rf.fit(X, y)

In [ ]:
# helper function for creating a feature importance dataframe
def imp_df(column_names, importances):
    df = pd.DataFrame({'feature': column_names,
                       'feature_importance': importances}) \
           .sort_values('feature_importance', ascending = False) \
           .reset_index(drop = True)
    return df

# plotting a feature importance dataframe (horizontal barchart)
def var_imp_plot(imp_df, title):
    imp_df.columns = ['feature', 'feature_importance']
    sns.barplot(x = 'feature_importance', y = 'feature', data = imp_df, orient = 'h', color = 'royalblue') \
       .set_title(title, fontsize = 20)

In [ ]:
base_imp = imp_df(X.columns, rf.feature_importances_)
top_30_imp = base_imp[0:30]
top_30_features = top_30_imp.feature

In [ ]:
pylab.rcParams["figure.figsize"] = (10,10)

In [ ]:
var_imp_plot(base_imp, 'Default feature importance (scikit-learn)')

In [ ]:
train_df_final_top_imp = OH_train_df[top_30_features].join(y)
X_test_final_top_imp = OH_X_test[top_30_features]

In [ ]:
train_df_final_top_imp.shape

In [ ]:
train_df_selected_features.shape

## 10. Export the final dfs

In the part *III. Model selection* I have tested different models with these 2 final sets of variables (80 vars and 30 vars). The result seems to be stable across all the models - the set of 80 variables we received after doing L1 regularization with logistic regression scores higher every time. I am thus using the selected_features dfs as the final input.

In [ ]:
train_df_selected_features.to_csv("train_df_final.csv", index=False)
X_test_selected_features.to_csv("X_test_final.csv", index=False)